# Library and Configuration

In [ ]:
# --- System & Environment Configuration --- 
import os
import random
from warnings import filterwarnings

# Matikan log TensorFlow yang tidak perlu
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# Abaikan warnings
filterwarnings("ignore")


# --- Core Library ---
import chardet
import numpy as np
import pandas as pd
from pathlib import Path

pd.set_option('display.max_colwidth', None)

# --- Visualization ---
import seaborn as sns
import matplotlib.pyplot as plt


# --- Scikit Learn ---
# Model Selection & Evaluation
from sklearn.model_selection import (
    KFold, 
    cross_validate, 
    cross_val_score, 
    train_test_split
)
from sklearn.metrics import (
    r2_score, 
    mean_squared_error,
    mean_absolute_error,
)
from sklearn.base import (
    BaseEstimator,
    TransformerMixin
)
    
# Preprocessing & Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder, 
    StandardScaler, 
    RobustScaler,
)
from sklearn.impute import (
    SimpleImputer
)
from sklearn.compose import (
    TransformedTargetRegressor
)

# ML Models
import optuna
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Possible Deep Learning Frameworks
import tensorflow as tf

ROOT_PATH = Path('/kaggle/input/preliminary-round-dac-prs-2024/dataset/')
TRAIN_PATH = ROOT_PATH/'train.csv'
WEATHER_PATH = ROOT_PATH/'Weather.csv'
TEST_PATH = ROOT_PATH/'test.csv'
METADATA_PATH = ROOT_PATH/'metadata.csv'
SAMPLE_SUBMISSION = ROOT_PATH/'sample_submission.csv'
SOLAR_IRRADIANCE = ROOT_PATH/'solar-irradiance'

TARGET = '% Baseline'
N_SPLITS=5

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

SEED = 42
set_seed(SEED)

print('library and configuration ready!')

In [ ]:
# read data to detect encoding
with open(METADATA_PATH, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

print(f"Detected encoding: {result['encoding']}")

# read with detected encoding
metadata = pd.read_csv(METADATA_PATH, encoding=result['encoding'], sep=';', index_col='No.')

# Utility Functions

In [ ]:
class SolarLunarTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        for col in self.features:
            # 1. Ekstrak menit (Row dengan NaT akan menghasilkan NaN di sini)
            minutes = X_copy[col].dt.hour * 60 + X_copy[col].dt.minute
            
            # 2. Buat Masking: Kita tandai mana baris yang NaN
            is_nan = minutes.isna()
            
            # 3. Fillna sementara dengan 0 agar np.sin tidak mengeluarkan Warning
            # Kita pakai .fillna(0) pada copy series agar tidak merusak data asli
            minutes_safe = minutes.fillna(0)
            
            # 4. Normalisasi & Kalkulasi
            normalized_time = minutes_safe / 1440.0
            
            sin_values = np.sin(2 * np.pi * normalized_time)
            cos_values = np.cos(2 * np.pi * normalized_time)
            
            # 5. PENTING: Kembalikan nilai NaN ke posisi aslinya menggunakan np.where
            # Logic: Jika is_nan True, isi dengan np.nan, jika False pakai hasil hitungan
            X_copy[f'{col}_sin'] = np.where(is_nan, np.nan, sin_values)
            X_copy[f'{col}_cos'] = np.where(is_nan, np.nan, cos_values)
            
            # Opsional: Drop kolom asli jika mau hemat memori
            # X_copy.drop(columns=[col], inplace=True)
            
        return X_copy[[c for c in X_copy.columns if '_sin' in c or '_cos' in c]]

# To make single time format for merging
def make_timestamp(df) -> pd.Series:  
    if "date_time" not in df.columns:
        raise KeyError("Column 'date_time' not found in dataframe.")
        
    dt = pd.to_datetime(df["date_time"])
    time = dt.dt.strftime("%b %-d, %Y %-I")
    suffix = dt.dt.strftime("%p").str.lower()
    
    timestamp = time + suffix
    
    return pd.to_datetime(timestamp)

# To convert multiple columns in dataframe to single datetime
def columns_to_datetime(df) -> pd.Series:
    return pd.to_datetime(
        df[['Year', 'Month', 'Day', 'Hour', 'Minute']]
    )

# To merge main dataset to multiple feature datasets
def merge_with_features(df, *feature_dfs) -> pd.DataFrame:
    if "Timestamp" not in df.columns:
        raise KeyError("Column 'Timestamp' not found in dataframe.")

    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="mixed")
        
    for feature in feature_dfs:
        df = df.merge(feature, on="Timestamp", how="left")
        
    return df

# To load multiple csv file by year
def load_yearly_csv(path, prefix, years) -> pd.DataFrame:
    return pd.concat(
        [pd.read_csv(path / f"{prefix}_{year}.csv") for year in years],
        ignore_index=True
    )

# To convert date time format to submission format
def convert_to_submission(df) -> pd.DataFrame:
    if "Timestamp" not in df.columns:
        raise KeyError("Column 'Timestamp' not found in dataframe.")
        
    dt = pd.to_datetime(df["Timestamp"])
    time = dt.dt.strftime("%b %-d, %Y %-I")
    suffix = dt.dt.strftime("%p").str.lower()

    df['Timestamp'] = time + suffix
    return df

# To plot target
def check_transformations(series, bins=50):
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))
    
    # --- Original Data ---
    sns.histplot(series, bins=bins, kde=True, ax=axes[0], color='skyblue', edgecolor='black')
    axes[0].set_title(f'Original\n(Skew: {series.skew():.2f})', fontsize=14)
    
    # --- Log Transformation ---
    series_log = np.log(series[series > 0]) 
    sns.histplot(series_log, bins=bins, kde=True, ax=axes[1], color='salmon', edgecolor='black')
    axes[1].set_title(f'Log Transform\n(Skew: {series_log.skew():.2f})', fontsize=14)
    
    # --- Square Root Transformation ---
    series_sqrt = np.sqrt(series)
    sns.histplot(series_sqrt, bins=bins, kde=True, ax=axes[2], color='lightgreen', edgecolor='black')
    axes[2].set_title(f'Square Root Transform\n(Skew: {series_sqrt.skew():.2f})', fontsize=14)

    plt.suptitle(f'{TARGET} Distribution', fontsize=18, fontweight='bold')
    plt.tight_layout()
    plt.show()

# To plot correlation coefficient for each feature with each others
def plot_numerical_correlation(X: pd.DataFrame, numerical_features: list) -> None:
    """
    Computes and visualizes the Pearson correlation matrix for numerical features
    using a heatmap.

    Args:
        X (pd.DataFrame): Input dataframe.
        numerical_features (list): List of numerical column names to correlate.
    """
    # Calculate the pairwise correlation of columns, excluding NA/null values
    correlation_matrix = X[numerical_features].corr(method='pearson')
    
    plt.figure(figsize=(15, 12))
    sns.heatmap(
        correlation_matrix,
        cmap='coolwarm',      # Diverging colormap (Red for pos, Blue for neg)
        vmin=-1, vmax=1,      # Anchor the colormap range
        center=0,             # Center the colormap at 0
        linewidths=.5,        
        cbar_kws={'label': 'Correlation Coefficient'}
    )
    
    plt.title('Numerical Features Correlation Heatmap (Pearson)', 
              fontsize=14, fontweight='bold')
    
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(rotation=0, fontsize=10)
    plt.tight_layout()
    plt.show()

def compare_similar_features(df, cols):
    """
    Membandingkan dua fitur dengan Boxplot berdampingan dan menghitung korelasi.
    Args:
        df (pd.DataFrame): Dataframe sumber.
        cols (list): List berisi dua nama kolom, e.g. ['tempC', 'Temperature']
    """
    # Validasi input biar gak error
    if len(cols) != 2:
        print("Error: List must be consist of two columns.")
        return
    
    col1, col2 = cols[0], cols[1]
    
    # Pearson Correlation
    corr_val = df[col1].corr(df[col2])

    # Plot Setup (1 Row, 2 Columns)
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Plot 1: First Boxplot
    sns.boxplot(y=df[col1], ax=axes[0], color='skyblue', width=0.4)
    axes[0].set_title(f'Distribusi: {col1}', fontsize=12, fontweight='bold')
    axes[0].grid(True, axis='y', alpha=0.3)

    # Plot 2: Second Boxplot
    sns.boxplot(y=df[col2], ax=axes[1], color='salmon', width=0.4)
    axes[1].set_title(f'Distribusi: {col2}', fontsize=12, fontweight='bold')
    axes[1].grid(True, axis='y', alpha=0.3)

    # Main title and correlation score
    plt.suptitle(f'Perbandingan Fitur: {col1} vs {col2}\nCorrelation Score: {corr_val:.4f}', 
                 fontsize=16, y=1.05)
    
    plt.tight_layout()
    plt.show()

# To add year, month, and day to solar and lunar features
def merge_column_with_timestamp(df, column) -> pd.Series:
    date_str = df['Timestamp'].dt.strftime('%Y-%m-%d')
    combined_str = date_str + ' ' + df[column]

    return pd.to_datetime(combined_str, errors='coerce')

# Data Loading

## Solar Irradiance features

In [ ]:
# Load 4 year solar irradiance datasets
solar_irradiance = load_yearly_csv(SOLAR_IRRADIANCE, 'Solar_Irradiance', range(2014, 2018))

# Make timestamp from 5 columns in solar_irradiance: ['Year', 'Month', 'Day', 'Hour', 'Minute']
solar_irradiance['date_time'] = columns_to_datetime(solar_irradiance)
solar_irradiance['Timestamp'] = make_timestamp(solar_irradiance)

# Drop unnecessary columns
solar_irradiance.drop(columns = ['date_time', 'Year', 'Month',
                                 'Day', 'Hour', 'Minute'], inplace=True)

solar_irradiance.head()

In [ ]:
solar_metadata = metadata[metadata['Variable Name'].isin(solar_irradiance.columns)].reset_index(drop=True)
solar_metadata

## Weather features

In [ ]:
# Load and make timestamp column for weather datasets
weather = pd.read_csv(WEATHER_PATH)
weather['Timestamp'] = make_timestamp(weather)

# Drop unnecessary column
weather.drop(columns='date_time', inplace=True)

# Merge timestamp for solar and lunar features
solar_lunar = ['sunrise', 'sunset', 'moonrise', 'moonset']

for feature in solar_lunar:
    weather[feature] = merge_column_with_timestamp(weather, feature)

weather.head()

In [ ]:
weather_metadata = metadata[metadata['Variable Name'].isin(weather.columns)].reset_index(drop=True)
weather_metadata

## Train and Test Merging

In [ ]:
# Load the train and test dataset
test  = pd.read_csv(TEST_PATH)
train = pd.read_csv(TRAIN_PATH)

# Define feature datasets
features = (weather, solar_irradiance)

# Merge main dataset with features
train_raw = merge_with_features(train, *features)
test_raw = merge_with_features(test, *features)

# Print train and test shape
print('Train shape:', train_raw.shape)
print('Test shape:', test_raw.shape) 

In [ ]:
train_metadata = metadata[metadata['Variable Name'].isin(train.columns)].reset_index(drop=True)
train_metadata

In [ ]:
train_raw.info()
test_raw.info() # Target feature: [% Baseline]

In [ ]:
train_raw['Cloud Type'].value_counts()

In [ ]:
print('===================================Train head===================================\n', train_raw.head())
print('\n===================================Test head===================================\n', test_raw.head())

In [ ]:
# from statsmodels.tools.tools import add_constant
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# X = train_df[numerical_features].copy()

# # Delete missing values
# X = (X.dropna()
#      # .drop(columns=['WindChillC']) # Opsional: delete highhest VIF value
#     )

# # intercept
# X_with_const = add_constant(X)

# # 4. Hitung VIF
# vif_data = pd.DataFrame()
# vif_data["feature"] = X_with_const.columns
# vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i)
#                    for i in range(len(X_with_const.columns))]

# # 5. Tampilkan hasil (urutkan dari yang terbesar)
# print(vif_data.sort_values(by="VIF", ascending=False))

# Exploratory Data Analysis

In [ ]:
# For a flexible exploration, we can separate features and target
X_raw=train_raw.drop(columns=[TARGET])
y_raw=train_raw[TARGET]

In [ ]:
check_transformations(y_raw)

Akan dilakukan square root transform

In [ ]:
numerical_features = X_raw.select_dtypes(np.number).columns

plot_numerical_correlation(X_raw, numerical_features)

Ada beberapa fitur yang berulang seperti `Temperature` dan `tempC` juga antara `DewPointC` dan `Dew Point`. Mengenai eliminasi fitur fitur tersebut akan dipertimbangkan nantinya 

## Exploration with similar features

In [ ]:
compare_similar_features(train_raw, ['Temperature', 'tempC'])
compare_similar_features(train_raw, ['pressure', 'Pressure'])
compare_similar_features(train_raw, ['DewPointC', 'Dew Point'])

In [ ]:
plt.figure(figsize=(15, 6))

# Scatter plot GHI vs Target
sns.scatterplot(x=train_raw['GHI'], y=train_raw[TARGET], alpha=0.3)
plt.title(f"Hubungan GHI vs {TARGET}")
plt.xlabel("GHI (Irradiance)")
plt.ylabel("Target (Output)")
plt.show()

In [ ]:
#  >>> SETTINGS <<<
# Ensure these columns exist. Assuming 'GNI' meant 'DNI' (Direct Normal Irradiance)
TARGET = '% Baseline'
FEATURES = ['GHI', 'DNI', 'Solar Zenith Angle', 'tempC']

def astronomical_investigation(df):
    data = df.copy()
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])
    
    # Extract time components
    data['Month'] = data['Timestamp'].dt.month
    data['Hour'] = data['Timestamp'].dt.hour
    
    # Create canvas
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Astronomical & Physical Data Inspection', fontsize=16)

    # ---------------------------------------------------------
    # PLOT 1: Monthly Production (Seasonality Check)
    # Purpose: Detect Hemisphere (North vs South)
    # ---------------------------------------------------------
    monthly_stats = data.groupby('Month')[[TARGET, 'GHI']].mean()
    
    ax1 = axes[0, 0]
    ax1_right = ax1.twinx()
    
    sns.lineplot(data=monthly_stats, x=monthly_stats.index, y=TARGET, 
                 ax=ax1, color='blue', marker='o', label='Energy Output', linewidth=2)
    sns.lineplot(data=monthly_stats, x=monthly_stats.index, y='GHI', 
                 ax=ax1_right, color='orange', marker='s', label='GHI (Irradiance)', linestyle='--')
    
    ax1.set_title("1. Monthly Average: Seasonality & Hemisphere")
    ax1.set_ylabel("Energy Output")
    ax1_right.set_ylabel("GHI")
    ax1.legend(loc='upper left')
    ax1_right.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)

    # ---------------------------------------------------------
    # PLOT 2: Solar Noon Verification (Timezone Check)
    # Purpose: Check if peak sun aligns with 12:00 PM
    # ---------------------------------------------------------
    hourly_stats = data.groupby('Hour')[[TARGET, 'GHI', 'DNI']].mean()
    
    ax2 = axes[0, 1]
    # Normalize values to 0-1 for easy comparison
    hourly_norm = (hourly_stats - hourly_stats.min()) / (hourly_stats.max() - hourly_stats.min())
    
    sns.lineplot(data=hourly_norm, x=hourly_norm.index, y=TARGET, ax=ax2, label='Energy Output')
    sns.lineplot(data=hourly_norm, x=hourly_norm.index, y='GHI', ax=ax2, label='GHI')
    sns.lineplot(data=hourly_norm, x=hourly_norm.index, y='DNI', ax=ax2, label='DNI (Direct)', linestyle=':')
    
    # Find peak hour
    peak_ghi_hour = hourly_stats['GHI'].idxmax()
    ax2.axvline(peak_ghi_hour, color='red', linestyle='--', alpha=0.5, label=f'Peak Sun ({peak_ghi_hour}:00)')
    
    ax2.set_title(f"2. Diurnal Cycle: Solar Noon Check (Peak at {peak_ghi_hour}:00?)")
    ax2.set_ylabel("Normalized Intensity (0-1)")
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # ---------------------------------------------------------
    # PLOT 3: Physical Correlation Heatmap
    # Purpose: Which astronomical factor drives energy most?
    # ---------------------------------------------------------
    corr_cols = [TARGET] + FEATURES
    corr_matrix = data[corr_cols].corr()
    
    ax3 = axes[1, 0]
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", ax=ax3)
    ax3.set_title("3. Physics Correlation Matrix")

    # ---------------------------------------------------------
    # PLOT 4: GHI vs DNI vs Energy (Scatter)
    # Purpose: Does energy follow Direct (DNI) or Global (GHI) radiation?
    # ---------------------------------------------------------
    ax4 = axes[1, 1]
    
    # Filter daytime only for clearer plot
    day_data = data[data['GHI'] > 10].sample(2000, random_state=42) # Sample to avoid lag
    
    sns.scatterplot(data=day_data, x='GHI', y=TARGET, ax=ax4, color='blue', alpha=0.3, label='GHI vs Energy')
    sns.scatterplot(data=day_data, x='DNI', y=TARGET, ax=ax4, color='orange', alpha=0.3, label='DNI vs Energy')
    
    ax4.set_title("4. Irradiance Impact: GHI vs DNI")
    ax4.set_xlabel("Irradiance (W/m2)")
    ax4.set_ylabel("Energy Output")
    ax4.legend()

    plt.tight_layout()
    plt.show()

    # --- TEXT REPORT ---
    print(">>> ASTRONOMICAL INSIGHTS <<<")
    
    # Seasonality Insight
    max_month = monthly_stats[TARGET].idxmax()
    print(f"1. Peak Month: {max_month}")
    if max_month in [11, 12, 1, 2]:
        print("   -> Insight: High production in Dec/Jan suggests SOUTHERN HEMISPHERE (Summer).")
    elif max_month in [6, 7, 8]:
        print("   -> Insight: High production in Jun/Jul suggests NORTHERN HEMISPHERE (Summer).")
        
    # Timezone Insight
    print(f"2. Solar Peak Hour: {peak_ghi_hour}:00")
    if peak_ghi_hour == 12:
        print("   -> Insight: PERFECT! Solar Noon aligns with Clock Noon. No timezone lag.")
    elif peak_ghi_hour > 12:
        print(f"   -> Insight: LAG DETECTED. Sun peaks late. Location might be WEST of timezone center.")
    else:
        print(f"   -> Insight: EARLY PEAK. Sun peaks early. Location might be EAST of timezone center.")

# Execute
astronomical_investigation(train_raw)

In [ ]:
def plot_monthly_distributions(df, target_col='% Baseline'):
    data = df.copy()
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])
    data['Month'] = data['Timestamp'].dt.month
    
    # Mapping angka bulan ke nama biar cantik
    month_map = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 
                 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['Month_Name'] = data['Month'].map(month_map)
    
    # Urutan bulan yang benar
    order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
             'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    # Setup Canvas
    fig, axes = plt.subplots(3, 1, figsize=(14, 18))
    
    # --- 1. TARGET (% BASELINE) ---
    sns.boxplot(data=data, x='Month_Name', y=target_col, order=order, ax=axes[0], palette='viridis')
    axes[0].set_title('1. Monthly Energy Distribution (Target)', fontsize=14)
    axes[0].set_ylabel('Energy Output (%)')
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # --- 2. GHI (GLOBAL HORIZONTAL IRRADIANCE) ---
    sns.boxplot(data=data, x='Month_Name', y='GHI', order=order, ax=axes[1], palette='Oranges')
    axes[1].set_title('2. Monthly Irradiance Distribution (GHI)', fontsize=14)
    axes[1].set_ylabel('GHI (W/m²)')
    axes[1].grid(True, alpha=0.3, axis='y')

    # --- 3. TEMPERATURE (TempC) ---
    sns.boxplot(data=data, x='Month_Name', y='tempC', order=order, ax=axes[2], palette='coolwarm')
    axes[2].set_title('3. Monthly Temperature Distribution', fontsize=14)
    axes[2].set_ylabel('Temperature (°C)')
    axes[2].grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.show()

# Eksekusi
plot_monthly_distributions(train_raw, target_col=TARGET)

In [ ]:
def inspect_physics_complete(df, sort_by='median'):
    data = df.copy()
    
    # 1. Data Preparation
    # Daytime only (GHI > 10) -> clouds at night don't matter
    day_data = data[data['GHI'] > 10].copy()
    
    # High Irradiance only (GHI > 200) -> for stability in efficiency calculation
    high_sun_data = data[data['GHI'] > 200].copy()
    high_sun_data['raw_efficiency'] = high_sun_data['% Baseline'] / high_sun_data['GHI']
    
    # Create Canvas (1 row, 3 columns)
    fig, axes = plt.subplots(1, 3, figsize=(24, 6))
    
    # --- PLOT 1: TEMPERATURE VS EFFICIENCY (Thermal) ---
    sns.scatterplot(
        data=high_sun_data, 
        x='tempC', y='raw_efficiency', 
        alpha=0.1, ax=axes[0], color='crimson'
    )
    sns.regplot(
        data=high_sun_data, 
        x='tempC', y='raw_efficiency', 
        scatter=False, ax=axes[0], color='black', line_kws={'linestyle':'--'}
    )
    axes[0].set_title('1. Thermal Physics: Temp vs Efficiency')
    axes[0].set_xlabel('Temperature (°C)')
    axes[0].set_ylabel('Raw Efficiency')
    axes[0].grid(True, alpha=0.3)

    # --- PLOT 2: CLOUD COVER VS ENERGY (Opacity) ---
    # Expectation: Negative Correlation. But does 100% cloud mean 0 energy?
    sns.scatterplot(
        data=day_data,
        x='cloudcover', y='% Baseline',
        alpha=0.05, ax=axes[1], color='dodgerblue' # low alpha to see density
    )
    # Add trend line
    sns.regplot(
        data=day_data,
        x='cloudcover', y='% Baseline',
        scatter=False, ax=axes[1], color='darkblue'
    )
    axes[1].set_title('2. Cloud Opacity: Coverage % vs Energy')
    axes[1].set_xlabel('Cloud Cover (%)')
    axes[1].set_ylabel('Energy Output (%)')
    axes[1].grid(True, alpha=0.3)

    # --- PLOT 3: CLOUD TYPE IMPACT (Categorical) ---
    # Logic for sorting
    if sort_by == 'Q3':
        order_metric = day_data.groupby('Cloud Type')['% Baseline'].quantile(0.75).sort_values(ascending=False)
    else:
        order_metric = day_data.groupby('Cloud Type')['% Baseline'].median().sort_values(ascending=False)
    
    sns.boxplot(
        data=day_data,
        x='Cloud Type', y='% Baseline',
        ax=axes[2], palette='Spectral',
        order=order_metric.index.tolist()
    )
    axes[2].set_title(f'3. Cloud Type Impact (Sorted by {sort_by})')
    axes[2].set_xticklabels(axes[2].get_xticklabels(), rotation=45, ha='right')
    axes[2].grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.show()

# Execute
inspect_physics_complete(train_raw, sort_by='median')

# Modeling

Data yang dilatih berupa data tabular dengan total fitur yang digunakan sebanyak 36 setelah dibersihkan. Karena kebanyakan fitur merupakan data numerik, maka akan digunakan model machine learning tradisional seperti XGBoost dan/atau LightGBM yang robust terhadap data numerikal dan outlier.

## Feature Engineering

In [ ]:
train_df = train_raw.copy()
test_df = test_raw.copy()

In [ ]:
def feature_engineering(df):
    df = df.copy()

    # Standardize timestamp
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = df.sort_values('Timestamp')

    # --- CLOUD MAPPING ---
    cloud_map = {
        'Unknown': 0, 'Opaque Ice': 1, 'Overlapping': 2, 'Super-Cooled Water': 3, 
        'Cirrus': 4, 'Fog': 5, 'Water': 6, 'Overshooting': 7, 
        'Probably Clear': 8, 'Clear': 9
    }
    df['Cloud Type'] = df['Cloud Type'].map(cloud_map).fillna(0)

    # --- PHYSICS CORRECTION (CRITICAL) ---
    # Observation: Solar peak occurs at 17:00 in raw data.
    # Adjustment: Shift time by -5 hours to align Solar Noon with 12:00.
    # This helps the model understand the true "shape" of the solar day.
    solar_time = df['Timestamp'] - pd.Timedelta(hours=5)
    solar_hour = solar_time.dt.hour
    doy = solar_time.dt.dayofyear

    # --- CYCLICAL FEATURES ---
    # Calculated on Solar Time for better alignment with physics
    df['hour_sin'] = np.sin(2 * np.pi * solar_hour / 24)
    df['hour_cos'] = np.cos(2 * np.pi * solar_hour / 24)
    
    # Monthly cycles
    df['month_sin'] = np.sin(2 * np.pi * df['Timestamp'].dt.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['Timestamp'].dt.month / 12)

    # --- ASTRONOMICAL FEATURES ---
    # Solar Declination (Cooper 1969)
    delta = 23.45 * np.sin(np.radians(360 * (284 + doy) / 365))
    df['solar_declination'] = delta

    # Equation of Time (EoT)
    B = np.radians((doy - 81) * 360 / 365)
    df['equation_of_time'] = 9.87 * np.sin(2*B) - 7.53 * np.cos(B) - 1.5 * np.sin(B)

    # Earth-Sun Distance Factor (Eccentricity)
    df['sun_earth_distance_factor'] = 1 + 0.033 * np.cos(np.radians(360 * doy / 365))
    df['extraterrestrial_radiation'] = 1367 * df['sun_earth_distance_factor']

    # --- SUNRISE / SUNSET LOGIC ---
    # Parsing raw time strings
    sunrise_dt = pd.to_datetime(df['sunrise'], format='%I:%M %p')
    sunset_dt = pd.to_datetime(df['sunset'], format='%I:%M %p')
    
    # Duration of daylight
    df['sunHour'] = (sunset_dt - sunrise_dt).dt.total_seconds()
    
    # Accurate daytime flag (comparing raw times)
    curr_time = df['Timestamp'].dt.time
    rise_time = sunrise_dt.dt.time
    set_time = sunset_dt.dt.time
    
    df['is_daytime'] = [
        1 if (r <= c <= s) else 0 
        for c, r, s in zip(curr_time, rise_time, set_time)
    ]

    # --- SOLAR PHYSICS RATIOS ---
    # Adding epsilon to avoid division by zero
    epsilon = 1e-6
    df['clearsky_index'] = df['GHI'] / (df['Clearsky GHI'] + epsilon)
    df['diffuse_fraction'] = df['DHI'] / (df['GHI'] + epsilon)
    
    # Wind Cooling Potential (using Kelvin)
    df['wind_cooling_potential'] = df['windspeedKmph'] / (df['tempC'] + 273.15)

    # --- TIME AWARE FEATURES (SMART MERGE) ---
    # Create lag features correctly handling gaps/jumps in data
    df['target_time_1h'] = df['Timestamp'] - pd.Timedelta(hours=1)
    
    lookup = df[['Timestamp', 'GHI', 'cloudcover']].copy()
    lookup.columns = ['ts_ref', 'GHI_lag1', 'cloudcover_lag1']
    
    df = df.merge(lookup, left_on='target_time_1h', right_on='ts_ref', how='left')
    
    # Rolling stats (3h window)
    indexer = df.set_index('Timestamp')
    df['GHI_rolling_mean_3h'] = indexer['GHI'].rolling('3h', min_periods=1).mean().values

    # Cleanup
    df.drop(columns=['target_time_1h', 'ts_ref'], inplace=True)
    
    features_to_fill = ['GHI_lag1', 'cloudcover_lag1', 'GHI_rolling_mean_3h']
    df[features_to_fill] = df[features_to_fill].fillna(0)
    
    return df

# Apply to datasets
train_df = feature_engineering(train_df)
test_df = feature_engineering(test_df)

In [ ]:
print(train_df.shape)
print(test_df.shape)

## Traditional Machine Learning

In [ ]:
def get_feature_groups(
    df, target_col, cyclical_cols, manual_drop_cols=None
):
    """
    Memisahkan kolom menjadi Numerical, Ordinal, dan Cyclical secara otomatis.
    """
    if manual_drop_cols is None:
        manual_drop_cols = ['tempC', 'DewPointC', 'Pressure'] # Default drop
        
    # Kolom yang pasti dibuang (Target + Cyclical mentah + Manual Drop)
    features_to_drop = [target_col] + cyclical_cols + manual_drop_cols
    
    # Numerical: Ambil semua angka, lalu buang yang masuk daftar drop
    num_features = df.select_dtypes(np.number).columns.difference(features_to_drop).tolist()
    # Ordinal: Hardcoded sesuai strategimu
    ord_features = ['Cloud Type']
    # Cyclical: Sesuai input
    cyc_features = cyclical_cols
    
    # Gabungkan semua untuk select X nanti
    all_selected = list(set(num_features + ord_features + cyc_features))
    
    print(f"📊 Features Summary:")
    print(f"   - Numerical : {len(num_features)}")
    print(f"   -   Ordinal : {len(ord_features)}")
    print(f"   -  Cyclical : {len(cyc_features)}")
    print(f"   -     TOTAL : {len(all_selected)}")
    
    return num_features, ord_features, cyc_features, all_selected

def build_preprocessor(num_feat, ord_feat, cyc_feat):
    """
    Menyusun ColumnTransformer agar rapi.
    """
    # Sub-pipeline untuk Cyclical
    cyclical_pipeline = Pipeline([
        ('cyclic_encoding', SolarLunarTransformer(features=cyc_feat))
    ])

    # Sub-pipeline untuk Numerical
    numerical_pipeline = Pipeline([
        ('numeric_imputer', SimpleImputer(strategy='mean')),
        ('numeric_scaler', StandardScaler())
    ])

    # Main Transformer
    preprocessor = ColumnTransformer(transformers=[
        ('ord', 'passthrough', ord_feat),
        ('cyc', cyclical_pipeline, cyc_feat),
        ('num', numerical_pipeline, num_feat),
    ])
    
    return preprocessor

def get_model(model_name='xgb', random_state=42):
    """
    Mengembalikan object model yang sudah dikonfigurasi parameternya.
    """
    # --- LightGBM Config ---
    lgbm_params = {
        'n_estimators': 5000,
        'learning_rate': 0.0325,
        'num_leaves': 50,
        'max_depth': -1,
        'min_child_samples': 20,
        'subsample': 0.65,
        'colsample_bytree': 0.85,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1,
        'objective': 'regression',
        'metric': 'rmse',
        'random_state': random_state,
        'n_jobs': 1,
        'verbose': -1
    }

    # --- XGBoost Config ---
    xgb_params = {
        'n_estimators': 2600, 
        'learning_rate': 0.029244, 
        'max_depth': 8, 
        'min_child_weight': 6, 
        'subsample': 0.6568, 
        'colsample_bytree': 0.8655, 
        'reg_alpha': 0.00458,
        'reg_lambda': 7.83e-05, 
        'n_jobs': 1, 
        'random_state': random_state,
    }

    if model_name == 'lgbm':
        return LGBMRegressor(**lgbm_params)
    elif model_name == 'xgb':
        return XGBRegressor(**xgb_params)
    else:
        raise ValueError("Model not supported. Choose 'xgb' or 'lgbm'")

# Setup Data & Features
num_cols, ord_cols, cyc_cols, sel_features = get_feature_groups(
    train_df, TARGET, solar_lunar
)

In [ ]:
def train_and_evaluate(
    model_name, X, y, preprocessor, 
    n_splits=5, seed=42, use_sqrt_target=False
):
    """
    Fungsi eksekusi utama: Build Pipeline -> CV -> Print Result
    """
    print('='*60)
    print(f"🚀 TRAINING STARTED: {model_name.upper()}")
    print('='*60)
    
    # 1. Ambil Model
    model = get_model(model_name, seed)
    
    # 2. Bungkus Model (Opsional: TransformedTargetRegressor untuk RMSE lebih stabil)
    if use_sqrt_target:
        # Kalau mau pakai teknik akar kuadrat target (Pipeline_sqrt)
        regressor = TransformedTargetRegressor(
            regressor=model,
            func=np.sqrt,
            inverse_func=np.square
        )
    else:
        regressor = model

    # 3. Buat Pipeline Akhir
    final_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', regressor)
    ])
    
    # 4. Cross Validation
    print(f"Running {n_splits}-Fold CV...")
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    
    results = cross_validate(
        final_pipeline, X, y, 
        cv=cv, 
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        return_train_score=False
    )
    
    # 5. Reporting
    scores = -results['test_score'] # Convert negative RMSE to positive
    
    print("-" * 60)
    for i, score in enumerate(scores):
        print(f"  Fold {i+1} MSE: {score:.6f}")
    
    print("-" * 60)
    print(f"🏆 {model_name.upper()} AVG MSE: {scores.mean():.6f} (+/- {scores.std():.6f})")
    print("-" * 60)
    
    return final_pipeline, scores.mean()

X = train_df[sel_features]
y = train_df[TARGET]

# Build Preprocessor
preprocessor = build_preprocessor(num_cols, ord_cols, cyc_cols)

wrapper = True
pipeline_xgb, score_xgb = train_and_evaluate(
    'xgb', X, y, preprocessor, N_SPLITS, SEED, use_sqrt_target=wrapper)

pipeline_lgbm, score_lgbm = train_and_evaluate(
    'lgbm', X, y, preprocessor, N_SPLITS, SEED, use_sqrt_target=wrapper)

In [ ]:
# optuna.logging.set_verbosity(optuna.logging.ERROR)

# def objective(trial):
#     param_grid = {
#         'n_estimators': trial.suggest_int('n_estimators', 1000, 3000, step=100),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
#         'max_depth': trial.suggest_int('max_depth', 5, 12),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
#         'subsample': trial.suggest_float('subsample', 0.6, 0.9),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
#         'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True), # L1 Regularization
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True), # L2 Regularization

#         'tree_method': 'hist',
#         'random_state': SEED,
#         'n_jobs': -1,
#         'device': 'cuda'
#     }

#     # Buat Model dengan parameter dari trial
#     model = XGBRegressor(**param_grid)

#     # Masukkan ke Pipeline
#     pipeline_optuna = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('model', model)
#     ])

#     # Cross Validation
#     cv = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
#     scores = cross_val_score(pipeline_optuna, X, y, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=1)
    
#     rmse = np.sqrt(-scores.mean())
#     return -scores.mean()
    
# study = optuna.create_study(direction='minimize') # Minimize error
# study.optimize(objective, n_trials=50, show_progress_bar=True)

# print('Best hyperparameters:', study.best_params)
# print('Best RMSE:', study.best_value)

In [ ]:
def plot_importance_final(pipeline, ord_cols, cyc_cols, num_cols, top_n=25):
    # --- 1. BUKA BUNGKUS MODEL (UNWRAPPER) ---
    # Ambil step 'model' dari pipeline
    wrapper = pipeline.named_steps['model']
    
    # Cek apakah dia TransformedTargetRegressor?
    if hasattr(wrapper, 'regressor_'):
        print("📦 Terdeteksi TransformedTargetRegressor. Mengambil inner model...")
        actual_model = wrapper.regressor_
    else:
        print("✅ Model tidak dibungkus (Standard).")
        actual_model = wrapper
        
    # --- 2. AMBIL NILAI IMPORTANCE ---
    if hasattr(actual_model, 'feature_importances_'):
        # Sklearn standard / XGBoost Scikit-Learn API
        importances = actual_model.feature_importances_
    elif hasattr(actual_model, 'booster_'): 
        # LightGBM Native API
        importances = actual_model.booster_.feature_importance(importance_type='gain')
    else:
        print("❌ Error: Model tidak memiliki atribut feature_importances_")
        return

    # --- 3. SUSUN NAMA FITUR (SESUAI URUTAN PREPROCESSOR) ---
    # Logika: Ordinal -> Cyclical (di-expand jadi sin/cos) -> Numerical
    
    # Expand Cyclical (karena pipeline cyclical memecah 1 kolom jadi 2)
    expanded_cyc_feat = []
    for feat in cyc_cols:
        expanded_cyc_feat.append(f"{feat}_sin")
        expanded_cyc_feat.append(f"{feat}_cos")
    
    # Gabungkan list nama
    final_names = list(ord_cols) + expanded_cyc_feat + list(num_cols)
    
    # --- 4. VALIDASI & PLOTTING ---
    print(f"📊 Model Features: {len(importances)}")
    print(f"📝 Feature Names : {len(final_names)}")
    
    if len(final_names) != len(importances):
        print("⚠️ Warning: Jumlah fitur tidak cocok! Menggunakan nama dummy.")
        final_names = [f"Feature_{i}" for i in range(len(importances))]
    else:
        print("✅ MATCH! Nama fitur sinkron.")

    # Buat DataFrame
    importance_df = pd.DataFrame({
        'Feature': final_names,
        'Importance': importances
    })

    # Sort & Plot
    importance_df = importance_df.sort_values(by='Importance', ascending=False).head(top_n)

    plt.figure(figsize=(10, 8))
    sns.barplot(x='Importance', y='Feature', data=importance_df, palette='viridis')
    plt.title(f'Top {top_n} Feature Importance (Wrapper Supported)')
    plt.tight_layout()
    plt.show()

In [ ]:
print("⏳ Training ulang model pada seluruh dataset...")
pipeline_xgb.fit(X, y) 
print("✅ Model selesai dilatih!")

# 2. BARU PLOT FEATURE IMPORTANCE
plot_importance_final(
    pipeline_xgb, 
    ord_cols, 
    cyc_cols, 
    num_cols
)

In [ ]:
print("⏳ Training ulang model pada seluruh dataset...")
pipeline_lgbm.fit(X, y) 
print("✅ Model selesai dilatih!")


plot_importance_final(
    pipeline_lgbm,
    ord_cols, 
    cyc_cols, 
    num_cols
)

In [ ]:
# --- 3. PREDIKSI KE DATA TEST ---
print("\n🔮 Predicting Test Data...")
# Pastikan pakai data test yang sudah di-feature engineering
X_test = test_df

pred_xgb = pipeline_xgb.predict(X_test)
pred_lgbm = pipeline_lgbm.predict(X_test)

# --- 4. BLENDING (Weighted Average) ---
# Kasih bobot 70% XGBoost (karena skornya udah dewa) + 30% LightGBM (buat jaga-jaga)
print("⚖️  Blending: 50% XGB + 50% LGBM")
final_pred = (0.45 * pred_xgb) + (0.55 * pred_lgbm)

# Safety: Gak boleh negatif
final_pred = np.maximum(final_pred, 0)

# --- 5. MASKING MALAM HARI (Natural Sunrise/Sunset) ---
print("🌑 Applying Night Masking...")

# Ambil jam sunrise/sunset dari data test
# Copy dulu biar gak ngerusak data asli
temp_test = test_df.copy()
temp_test['Timestamp'] = pd.to_datetime(temp_test['Timestamp'])

# Convert string jam ke object time
sunrise_dt = pd.to_datetime(temp_test['sunrise'], format='%I:%M %p').dt.time
sunset_dt = pd.to_datetime(temp_test['sunset'], format='%I:%M %p').dt.time
current_time = temp_test['Timestamp'].dt.time

# Logika: Kalau jam skrg < sunrise ATAU jam skrg > sunset -> NOL
is_night = [
    (curr < rise) or (curr > set_) 
    for curr, rise, set_ in zip(current_time, sunrise_dt, sunset_dt)
]

# Eksekusi masking
# final_pred[is_night] = 0

# --- 6. SAVE SUBMISSION ---
submission = pd.read_csv(SAMPLE_SUBMISSION)
submission['% Baseline'] = final_pred
submission.to_csv('submission.csv', index=False)

print("\nFile saved in submission.csv")
display(submission.head())